In [36]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import keras
from keras import layers, models, callbacks, optimizers, regularizers
import matplotlib.pyplot as plt


In [37]:
OUTPUT_DIR = "../outputs/"
os.makedirs(OUTPUT_DIR, exist_ok=True)
MODEL_DIR = os.path.join(OUTPUT_DIR, "models")
os.makedirs(MODEL_DIR, exist_ok=True)
SUBMISSION_DIR = os.path.join(OUTPUT_DIR, "submissions")
os.makedirs(SUBMISSION_DIR, exist_ok=True)

In [38]:
# Load data
PROCESSED_DATA_DIR = "../data/processed/"
train_df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "train_processed.csv"))
test_df = pd.read_csv(os.path.join(PROCESSED_DATA_DIR, "test_processed.csv"))

# Separate target
y = train_df["Survived"]
X = train_df.drop(columns=["Survived", "PassengerId"])

# Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42, stratify=y)
print(f"Train shape: {X_train.shape}, Val shape: {X_val.shape}")
scaler = StandardScaler()
num_features = ['Age', 'Fare', 'FamilySize']

X_train[num_features] = scaler.fit_transform(X_train[num_features])
X_val[num_features] = scaler.transform(X_val[num_features])
test_df[num_features] = scaler.transform(test_df[num_features])

Train shape: (801, 22), Val shape: (90, 22)


In [39]:
print(X_train.head(10).to_markdown())

|     |   Pclass |        Age |   SibSp |   Parch |        Fare |   FamilySize |   IsAlone |   HasCabin | Sex_male   | Embarked_Q   | Embarked_S   | Title_Miss   | Title_Mr   | Title_Mrs   | Title_Rare   | Cabin_deck_B   | Cabin_deck_C   | Cabin_deck_D   | Cabin_deck_E   | Cabin_deck_F   | Cabin_deck_G   | Cabin_deck_T   |
|----:|---------:|-----------:|--------:|--------:|------------:|-------------:|----------:|-----------:|:-----------|:-------------|:-------------|:-------------|:-----------|:------------|:-------------|:---------------|:---------------|:---------------|:---------------|:---------------|:---------------|:---------------|
|  86 |        3 | -1.00223   |       1 |       3 |  0.051433   |    1.9028    |         0 |          0 | True       | False        | True         | False        | True       | False       | False        | False          | False          | False          | False          | True           | False          | False          |
| 329 |        1 | -1.002

In [40]:
# Convert to tensors
X_train = tf.convert_to_tensor(X_train, dtype=tf.float32)
y_train = tf.convert_to_tensor(y_train, dtype=tf.float32)
X_val = tf.convert_to_tensor(X_val, dtype=tf.float32)
y_val = tf.convert_to_tensor(y_val, dtype=tf.float32)
passenger_ids = test_df["PassengerId"]
X_test = tf.convert_to_tensor(test_df.drop(columns=["PassengerId","Survived"]), dtype=tf.float32)

# --- Build DNN Model (Functional API) ---
inputs = tf.keras.Input(shape=(X_train.shape[1],))
x = layers.Dense(128, activation='relu',name='l1',kernel_regularizer=regularizers.l2(0.001))(inputs)
x = layers.Dropout(0.2,name='l1_dropout')(x)
x = layers.Dense(32, activation='relu',name='l2',kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.1,name='l2_dropout')(x)
x = layers.Dense(16, activation='relu',name='l3',kernel_regularizer=regularizers.l2(0.001))(x)
x = layers.Dropout(0.1,name='l3_dropout')(x)
x = layers.Dense(8, activation='relu',name='l4')(x)
outputs = layers.Dense(1, activation='sigmoid',name='output')(x)

model = models.Model(inputs, outputs, name="DNN_Model")

In [41]:
model.summary(print_fn=lambda x: print(x))

Model: "DNN_Model"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 22)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l1 (Dense)                      │ (None, 128)            │         2,944 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l1_dropout (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l2 (Dense)                      │ (None, 32)             │         4,128 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ l2_dropout (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼──────────────────────

In [42]:
initial_lr = 0.001
optimizer = optimizers.Adam(learning_rate=initial_lr)

# Define LR scheduler (reduce when plateau)
lr_scheduler = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

# Define Early Stopping
early_stop = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=35,
    restore_best_weights=True,
    verbose=1
)

if os.path.exists(os.path.join(MODEL_DIR, f"{model.name.replace(' ', '_')}.keras")):
    model= keras.models.load_model(os.path.join(MODEL_DIR, f"{model.name.replace(' ', '_')}.keras"))
    print("Loaded existing model weights.")
    # Compile model
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
else:
    print("No existing model weights found, starting fresh.")
    # Compile model
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    history = model.fit(
        X_train,
        y_train,
        validation_split=0.2,
        epochs=100,
        batch_size=32,
        callbacks=[lr_scheduler, early_stop],
        verbose=1
    )
    plt.plot(history.history['accuracy'], label='Train Acc')
    plt.plot(history.history['val_accuracy'], label='Val Acc')
    plt.legend()
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.title('Training vs Validation Accuracy')
    plt.show()


Loaded existing model weights.


In [43]:
if not os.path.exists(os.path.join(MODEL_DIR, f"{model.name.replace(' ', '_')}.keras")):
    model.save(os.path.join(MODEL_DIR, f"{model.name.replace(' ', '_')}.keras"))
    print(f"Saved model for {model.name}")
    

In [44]:
# --- Evaluate on Validation Set ---
val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
print(f"\nValidation Accuracy: {val_acc:.4f}")


Validation Accuracy: 0.7889


In [45]:
# --- Predict on Test Data ---
test_preds = model.predict(X_test, verbose=1)
test_preds = (test_preds > 0.5).astype(int).flatten()  # Convert sigmoid outputs to 0/1

# --- Create Submission DataFrame ---
submission = pd.DataFrame({
    "PassengerId": passenger_ids,
    "Survived": test_preds
})

# --- Save submission file ---
submission_path = os.path.join(SUBMISSION_DIR, f"{model.name}_submission.csv")
submission.to_csv(submission_path, index=False)
print(f"Submission file saved to: {submission_path}")

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
Submission file saved to: ../outputs/submissions\DNN_Model_submission.csv
